In [ ]:
from spyre import server
import __main__ as main
main.__file__ = "main_file"

class SimpleApp(server.App):
    title = "Simple App"
    inputs = [dict( type='text',
                    key='words',
                    label='write words here',
                    value='hello world',
                    action_id='simple_html_output')]

    outputs = [dict( type='html',
                     id='simple_html_output')]

    def getHTML(self, params):
        words = params["words"]
        return "Here's what you wrote in the textbox: <b>%s</b>" % words

app = SimpleApp()
app.launch()

<h2 align="center"> Лабораторна робота №3</h2>
<p align="center">ФБ-23 Ткачук Андрій</p>

## Хід виконання:

### Створити веб-додаток із використанням модуля Spyre, який дозволить: обрати часовий ряд VCI, TCI, VHI для набору даних із лабораторної роботи 1 (випадаючий список);

## Завдання лабораторної роботи:
<ul>
    <li>Створити ще один випадаючий список для вибору області </li>
    <li> Створити текстове поле для введення інтервалу місяців</li>
    <li> Створити таблицю та графік для відображення даних</li>
</ul>

Оновлення графіку можна реалізувати 2 способами:
<ul>
<li>В inputs додати action_id зі значенням id outputs</li>
<li>В outputs додати control_id зі значенням id controls</li>

</ul>

### Основна частина:


In [ ]:
from df_creation import dir_c
#Завантажуються дані з другої лаб. роботи
dir_c()

In [ ]:
from spyre import server
from df_creation import cleandf
import seaborn as sns
from matplotlib import pyplot as plt

import __main__ as main
main.__file__ = "main_file"

server.include_df_index = True


class StockExample(server.App):
    title = "NOAA DATA"

######### 1.Дані VCI,TCI,VHI з 1 лабораторної у вигляді випадаючого списку
    inputs = [{
            "type": 'dropdown',
            "label": 'Values',
            "options": [{"label": "VCI", "value": "VCI"},
                        {"label": "TCI", "value": "TCI"},
                        {"label": "VHI", "value": "VHI"}],
            "key": 'choosen_value',
            #"action_id" : "plot_id"
        },
#########  2. Вибрати область, для якої буде виконуватись аналіз (випадаючий список);
        {
            "type": 'dropdown',
            "label": 'Values',
            "options": [{"label": "Вінницька", "value": "1"},
                        {"label": "Волинська", "value": "2"},
                        {"label": "Дніпропетровська", "value": "3"},
                        {"label": "Донецька", "value": "4"},
                        {"label": "Житомирська", "value": "5"},
                        {"label": "Закарпатська", "value": "6"},
                        {"label": "Запорізька", "value": "7"},
                        {"label": "Івано-Франківська", "value": "8"},
                        {"label": "Київська", "value": "9"},
                        {"label": "Кіровоградська", "value": "10"},
                        {"label": "Луганська", "value": "11"},
                        {"label": "Львівська", "value": "12"},
                        {"label": "Миколаївська", "value": "13"},
                        {"label": "Одеська", "value": "14"},
                        {"label": "Полтавська", "value": "15"},
                        {"label": "Рівненська", "value": "16"},
                        {"label": "Сумська", "value": "17"},
                        {"label": "Тернопільська", "value": "18"},
                        {"label": "Харківська", "value": "19"},
                        {"label": "Херсонська", "value": "20"},
                        {"label": "Хмельницька", "value": "21"},
                        {"label": "Черкаська", "value": "22"},
                        {"label": "Чернівецька", "value": "23"},
                        {"label": "Чернігівська", "value": "24"},
                        {"label": "Республіка Крим", "value": "25"},
                        {"label": "Київ", "value": "26"},
                        {"label": "Севастополь", "value": "27"},],
            "key": 'province_value',
        #"action_id" : "plot_id"
        },
######### 3.Створити текстове поле для введення інтервалу тижнів
        {
            "type":"text",
            "label":"weeks range",
            "key":"weeks",
            "value":"9-15"
           #"action_id":"table_id"
        }
        ]

    outputs = [{
            "type": "plot",
            "id": "plot_id",
            "tab": "Plot",
            "control_id": "button1"
        },
        {
            "type":"table",
            "id":"table_id",
            "tab": "Table",
            "control_id": "button1"
        }]

    controls = [{
            "type": "button",
            "id": "button1",
            "label": "Update data"
        }]
    
    tabs=['Plot','Table']

    def getData(self,params):
        dataf = cleandf()
        ch_province=params["province_value"]
        ch_value=params['choosen_value']

        
        week_range=params['weeks'].split('-')
        if int(week_range[0])>int(week_range[1]):
            t=int(week_range[0])
            week_range[0]=int(week_range[1])
            week_range[1]=t
        else:
            week_range[0]=int(week_range[0])
            week_range[1]=int(week_range[1])

        cdf=dataf[(dataf['ID'].astype("str")==ch_province)&(dataf['week'] > week_range[0]-1)&(dataf['week'] < week_range[1]+1) ][['year','week','SMN','SMT',ch_value,'ID']]
        return cdf
    
    def getPlot(self, params):
        cdf = self.getData(params)
        cdf['week']=cdf['week'].astype('int64')
        ch_value=params['choosen_value']
        plt.figure(figsize=(10,6))
        plt_obj=sns.lineplot(x=cdf['week'], y=cdf[ch_value])
        plt_obj.set_xticks(range(int(min(cdf['week'])), int(max(cdf['week'])) + 1))
        return plt_obj
app = StockExample()
app.launch(port=9093)

### Версія зі слайдером

In [ ]:
from spyre import server
from df_creation import cleandf
import seaborn as sns
from matplotlib import pyplot as plt
import __main__ as main
main.__file__ = "main_file"

server.include_df_index = True


class StockExample(server.App):
    title = "NOAA DATA"

######### 1.Дані VCI,TCI,VHI з 1 лабораторної у вигляді випадаючого списку
    inputs = [{
            "type": 'dropdown',
            "label": 'Values',
            "options": [{"label": "VCI", "value": "VCI"},
                        {"label": "TCI", "value": "TCI"},
                        {"label": "VHI", "value": "VHI"}],
            "key": 'choosen_value',
            #"action_id" : "plot_id"
        },
        #Обираю рік за допомогою слайдера
        {
            "type":'slider',
            "label": 'Year',
            "key": 'year_slider',
            "value" :1982,
            "min" : 1982,
            "max" : 2024,
            #"action_id" : "plot_id"
        },
#########  2. Вибрати область, для якої буде виконуватись аналіз (випадаючий список);
        {
            "type": 'dropdown',
            "label": 'Values',
            "options": [{"label": "Вінницька", "value": "1"},
                        {"label": "Волинська", "value": "2"},
                        {"label": "Дніпропетровська", "value": "3"},
                        {"label": "Донецька", "value": "4"},
                        {"label": "Житомирська", "value": "5"},
                        {"label": "Закарпатська", "value": "6"},
                        {"label": "Запорізька", "value": "7"},
                        {"label": "Івано-Франківська", "value": "8"},
                        {"label": "Київська", "value": "9"},
                        {"label": "Кіровоградська", "value": "10"},
                        {"label": "Луганська", "value": "11"},
                        {"label": "Львівська", "value": "12"},
                        {"label": "Миколаївська", "value": "13"},
                        {"label": "Одеська", "value": "14"},
                        {"label": "Полтавська", "value": "15"},
                        {"label": "Рівенська", "value": "16"},
                        {"label": "Сумська", "value": "17"},
                        {"label": "Тернопільська", "value": "18"},
                        {"label": "Харківська", "value": "19"},
                        {"label": "Херсонська", "value": "20"},
                        {"label": "Хмельницька", "value": "21"},
                        {"label": "Черкаська", "value": "22"},
                        {"label": "Чернівецька", "value": "23"},
                        {"label": "Чернігівська", "value": "24"},
                        {"label": "Республіка Крим", "value": "25"},
                        {"label": "Київ", "value": "26"},
                        {"label": "Севастополь", "value": "27"},],
            "key": 'province_value',
        #"action_id" : "plot_id"
        },
######### 3.Створити текстове поле для введення інтервалу тижнів
        {
            "type":"text",
            "label":"weeks range",
            "key":"weeks",
            "value":"9-15"
           #"action_id":"table_id"
        }
        ]

    outputs = [{
            "type": "plot",
            "id": "plot_id",
            "tab": "Plot",
            "control_id": "button1"
            },
            {
                "type":"table",
                "id":"table_id",
                "tab": "Table",
                "control_id": "button1"
            }]

    controls = [{
        "type": "button",
        "id": "button1",
        "label": "Update data"
    }]

    tabs=['Plot','Table']
 
    def getData(self,params):
        dataf = cleandf()
        ch_province=params["province_value"]
        ch_value=params['choosen_value']
        ch_year = params['year_slider']
        
        week_range=params['weeks'].split('-')
        if int(week_range[0])>int(week_range[1]):
            t=int(week_range[0])
            week_range[0]=int(week_range[1])
            week_range[1]=t
        else:
            week_range[0]=int(week_range[0])
            week_range[1]=int(week_range[1])

        cdf=dataf[(dataf['year'].astype('int64') == ch_year) &(dataf['ID'].astype("str")==ch_province)&(dataf['week'] > week_range[0]-1)&(dataf['week'] < week_range[1]+1) ][['year','week','SMN','SMT',ch_value,'ID']]
        return cdf
    
    def getPlot(self, params):
        cdf = self.getData(params)
        cdf['week']=cdf['week'].astype('int64')
        ch_value=params['choosen_value']
        plt.figure(figsize=(14,6))
        plt_obj=sns.lineplot(x=cdf['week'], y=cdf[ch_value])
        plt_obj.set_xticks(range(int(min(cdf['week'])), int(max(cdf['week'])) + 1))
        return plt_obj
app = StockExample()
app.launch(port=9093)